# Welcome to Jupyter!

This repo contains an introduction to [Jupyter](https://jupyter.org) and [IPython](https://ipython.org).

Outline of some basics:

* [Notebook Basics](../examples/Notebook/Notebook%20Basics.ipynb)
* [IPython - beyond plain python](../examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb)
* [Markdown Cells](../examples/Notebook/Working%20With%20Markdown%20Cells.ipynb)
* [Rich Display System](../examples/IPython%20Kernel/Rich%20Output.ipynb)
* [Custom Display logic](../examples/IPython%20Kernel/Custom%20Display%20Logic.ipynb)
* [Running a Secure Public Notebook Server](../examples/Notebook/Running%20the%20Notebook%20Server.ipynb#Securing-the-notebook-server)
* [How Jupyter works](../examples/Notebook/Multiple%20Languages%2C%20Frontends.ipynb) to run code in different languages.

You can also get this tutorial and run it on your laptop:

    git clone https://github.com/ipython/ipython-in-depth

Install IPython and Jupyter:

with [conda](https://www.anaconda.com/download):

    conda install ipython jupyter

with pip:

    # first, always upgrade pip!
    pip install --upgrade pip
    pip install --upgrade ipython jupyter

Start the notebook in the tutorial directory:

    cd ipython-in-depth
    jupyter notebook

In [1]:
import sys
!{sys.executable} -m pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import numpy as np
import datetime
import func_lib

endpoint_url = "https://query.wikidata.org/sparql"
item = "item"


class Relation:
    """
    The class returned when createRelation is called.
    It contains string field with query.
    We call Relation.query when we need to do the query.
    """

    def __init__(self, entity_id: str, property_id: str, isSubject: bool, rowVerbose: bool,
                 colVerbose: bool, time_property: str, time: str, name: str, label: bool, limit=10000):
        self.entity_id = entity_id
        self.query_str = ""
        self.dic = {}
        self.result_dic = {"Entity ID": []}
        self.df = pd.DataFrame()
        self.count = 0
        self.time_property = time_property
        self.time = time
        self.limit = limit
        self.focus = "Entity ID"
        if property_id:
            self.extend(property_id, isSubject, name, rowVerbose, colVerbose, limit, time_property, time, label)

    def generate_html(self, name: str):
        html = (self.df).to_html()
        text_file = open(name, "w", encoding='utf-8')
        text_file.write(html)
        text_file.close()

    def query(self, require=None):
        if self.query_str == "":
            self.result_dic = {"Entity ID": ['http://www.wikidata.org/entity/' + str(self.entity_id)]}
            return self.result_dic
        results = get_results(endpoint_url, self.query_str)
        result_dict = {"Entity ID": ['http://www.wikidata.org/entity/' + str(self.entity_id)]}
        for i in range(1, self.count + 1):
            result_dict[self.dic[i]["name"] + '_' + self.dic[i]['property_id']] = []
            if self.dic[i]["colVerbose"]:
                result_dict[self.dic[i]["name"] + '_rank_' + self.dic[i]['property_id'] + '_rank'] = []
                for key, value in self.dic[i]["property_name_dic"].items():
                    result_dict[
                        self.dic[i]["name"] + "_" + value + '_' + self.dic[i]['property_id'] + '_' + str(key)] = []
                for key, value in self.dic[i]["ref_dic"].items():
                    result_dict[self.dic[i]["name"] + "_ref_" + self.dic[i]['property_id'] + '_' + str(key)] = []

            if self.dic[i]["label"]:
                result_dict[self.dic[i]["name"] + '_' + self.dic[i]['property_id'] + 'Label'] = []

        for result in results['results']['bindings']:
            for key, value in result_dict.items():
                if key in result.keys():
                    result_dict[key].append(result[key]['value'])
                else:
                    result_dict[key].append('NA')
        result_dict["Entity ID"] = ['http://www.wikidata.org/entity/' + str(self.entity_id)] * len(
            result_dict[self.dic[self.count]["name"] + '_' + self.dic[self.count]["property_id"]])
        self.result_dic = result_dict
        self.df = pd.DataFrame.from_dict(self.result_dic)
        for i in range(1, self.count + 1):
            if self.dic[i]["colVerbose"] and not self.dic[i]["rowVerbose"]:
                col = self.dic[i]['name'] + '_rank_' + self.dic[i]['property_id'] + '_rank'
                if any(self.df[col] == 'http://wikiba.se/ontology#PreferredRank'):
                    self.df = self.df.loc[self.df[col] == 'http://wikiba.se/ontology#PreferredRank']
                else:
                    self.df = self.df.loc[self.df[col] == 'http://wikiba.se/ontology#NormalRank']
#         if require is not None:
#             for r in require:
#                 self.df = self.df.loc[self.df[r] != 'NA']
        self.df = pd.DataFrame(data=self.df)
#         if self.df.shape[0] >= 10000:
#             print("Warning: Your query leads to too many results. Only 10,000 returned.")
        return self.df

    def extend(self, property_id: str, isSubject: bool, name: str, rowVerbose=False, colVerbose=False, limit=None,
               time_property=None, time=None, search=None, label=False):
        self.count += 1
        self.dic[self.count] = {}
        self.dic[self.count]["name"] = name
        self.dic[self.count]["focus"] = self.focus
        self.dic[self.count]["property_id"] = property_id
        self.dic[self.count]["isSubject"] = isSubject
        self.dic[self.count]["limit"] = limit
        self.dic[self.count]["rowVerbose"] = rowVerbose
        self.dic[self.count]["colVerbose"] = colVerbose
        self.dic[self.count]['time_property'] = time_property
        self.dic[self.count]['time'] = time
        self.dic[self.count]['search'] = search
        self.dic[self.count]['label'] = label
        if rowVerbose or colVerbose:
            self.dic[self.count]["property_name_dic"], self.dic[self.count][
                "ref_dic"] = self.search_property_for_verbose()
        if time_property and time:
            self.time_property = time_property
            self.time = time
        if limit:
            self.limit = limit
        self.query_str = self.define_query_relation()

    def changeFocus(self, name="Entity ID"):
        self.focus = name
        
    def applyFunction(self, objcolumn, func, name):
        if type(func) == str:
            if func.startswith('F'):
                try:
                    func_id = int(func[1:])
                    if func_id == 0:
                        self.df[name] = self.df[objcolumn]
                    else:
                        if func_id >= func_lib.func_num():
                            print("Not available.")
                        else:
                            self.df[name] = self.df[objcolumn].apply(func_lib.func_list[func_id])
                except:
                    raise Exception("Not a valid function id, a valid function id should be 'Fn', n is an integer.")
            else:
                raise Exception("Not a valid function id, a valid function id should be 'Fn', n is an integer.")
        else:
            self.df[name] = self.df[objcolumn].apply(func)

    def define_query_relation(self):
        rdf_triple, time_filter, limit_statement = """""", """""", """"""
        if self.count < 1:
            return None
        focusChanges = 0
        for i in range(1, self.count + 1):
            if self.dic[i]["rowVerbose"] or self.dic[i]["colVerbose"]:
                if self.dic[i]["search"] is None and not self.dic[i]["isSubject"]:
                        rdf_triple += """OPTIONAL {"""
                if self.dic[i]["focus"] == "Entity ID":
#                     if self.dic[i]["search"] is None:
#                         rdf_triple += """OPTIONAL {"""
                    rdf_triple += """wd:""" + self.entity_id + """ p:""" + self.dic[i][
                        'property_id'] + """ ?statement_""" + str(i) + """. """ \
                                  + """?statement_""" + str(i) + """ ps:""" + self.dic[i][
                                      'property_id'] + """ ?""" + \
                                  self.dic[i]['name'] \
                                  + """_""" + self.dic[i]['property_id'] + """. """
                else:
                    rdf_triple += """?""" + self.dic[i]["focus"] + """ p:""" + self.dic[i][
                        'property_id'] + """ ?statement_""" + str(i) + """. """ \
                                  + """?statement_""" + str(i) + """ ps:""" + self.dic[i][
                                      'property_id'] + """ ?""" + \
                                  self.dic[i]['name'] \
                                  + """_""" + self.dic[i]['property_id'] + """. """
                for key, value in self.dic[i]["property_name_dic"].items():
                    rdf_triple += """OPTIONAL { """ + """?statement_""" + str(i) + """ pq:""" + str(key) \
                                  + """ ?""" + self.dic[i]['name'] + """_""" + value + """_""" + self.dic[i][
                                      'property_id'] + """_""" + str(key) + """.} """
                for key, value in self.dic[i]["ref_dic"].items():
                    rdf_triple += """OPTIONAL { ?statement_""" + str(
                        i) + """ prov:wasDerivedFrom ?refnode_""" + str(
                        i) + """. ?refnode_""" + str(i) \
                                  + """ pr:""" + str(key) + """ ?""" + self.dic[i]['name'] + """_ref_""" + \
                                  self.dic[i][
                                      'property_id'] + """_""" + str(key) + """.} """
                rdf_triple += """OPTIONAL { ?statement_""" + str(i) + """ wikibase:rank ?""" + self.dic[i][
                    'name'] + """_rank_""" + self.dic[i]['property_id'] + """_rank. } """
            # none-verbose version
            else:
                if self.dic[i]["focus"] == "Entity ID":
                    if self.dic[i]["isSubject"]:
#                         if self.dic[i]["search"] is None:
#                             rdf_triple += """OPTIONAL {"""
                        rdf_triple += """?""" + self.dic[i]["name"] + """_""" + self.dic[i][
                            'property_id'] + """ wdt:""" + self.dic[i][
                                          "property_id"] + """ wd:""" + self.entity_id + """. """
                    else:
                        if self.dic[i]["search"] is None:
                            rdf_triple += """OPTIONAL {"""
                        rdf_triple += """wd:""" + self.entity_id + """ wdt:""" + self.dic[i][
                            "property_id"] + """ ?""" + \
                                      self.dic[i]["name"] + """_""" + self.dic[i]['property_id'] + """. """
                else:
                    if self.dic[i]["isSubject"]:
#                         if self.dic[i]["search"] is None:
#                             rdf_triple += """OPTIONAL {"""
                        rdf_triple += """?""" + self.dic[i]["name"] + """_""" + self.dic[i][
                            'property_id'] + """ wdt:""" + self.dic[i]["property_id"] + """ ?""" + self.dic[i][
                                          'focus'] + """. """
                    else:
                        if self.dic[i]["search"] is None:
                            rdf_triple += """OPTIONAL {"""
                        rdf_triple += """?""" + self.dic[i]['focus'] + """ wdt:""" + self.dic[i][
                            "property_id"] + """ ?""" + self.dic[i]["name"] + """_""" + self.dic[i][
                                          'property_id'] + """. """
            if not self.dic[i]["isSubject"]:
                if i < self.count and self.dic[i]["focus"] != self.dic[i + 1]["focus"] and self.dic[i]["search"] is None:
                    focusChanges += 1
                elif self.dic[i]["search"] is None:
                    rdf_triple += """} """
        for i in range(focusChanges):
            rdf_triple += """} """
        for i in range(1, self.count + 1):
            if self.dic[i]['search'] is not None and self.dic[i]["search"] != '!NA':
                if isinstance(self.dic[i]['search'], tuple):
                    if isinstance(self.dic[i]['search'][0], str):
                        rdf_triple += """FILTER (YEAR(?""" + self.dic[i]['name'] + """_""" + self.dic[i][
                            'property_id'] + """) >= """ + \
                                      self.dic[i]['search'][0] + """ && YEAR(?""" + self.dic[i]['name'] + \
                                      """_""" + self.dic[i]['property_id'] + """) <= """ + self.dic[i]['search'][
                                          1] + """) """
                    else:
                        rdf_triple += """FILTER (?""" + self.dic[i]['name'] + """_""" + self.dic[i]['property_id'] + \
                                      """ >= """ + str(self.dic[i]['search'][0]) + """ && ?""" + self.dic[i]['name'] + \
                                      """_""" + self.dic[i]['property_id'] + """ <= """ + str(
                            self.dic[i]['search'][1]) + """) """
                else:
                    rdf_triple += """FILTER (?""" + self.dic[i]['name'] + """_""" + self.dic[i][
                        'property_id'] + """ = """ + \
                                  """wd:""" + self.dic[i]['search'] + """) """
        if self.time_property is not None:
            time_filter = """?""" + self.dic[1]["name"] + """ p:""" + self.time_property + """ ?pubdateStatement.	
                          ?pubdateStatement ps:""" + self.time_property + """ ?date	
                          FILTER (YEAR(?date) = """ + self.time + """)"""
        if self.limit is not None:
            limit_statement = """LIMIT """ + str(self.limit)
        label_statement = """Service wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }"""
        query = """SELECT DISTINCT"""
        for i in range(1, self.count + 1):
            if self.dic[i]["rowVerbose"] or self.dic[i]["colVerbose"]:
                query += """ ?""" + self.dic[i]["name"] + """_""" + self.dic[i]['property_id']
                if self.dic[i]["label"]:
                    query += """ ?""" + self.dic[i]["name"] + """_""" + self.dic[i]['property_id'] + """Label"""
                for key, value in self.dic[i]["property_name_dic"].items():
                    query += """ ?""" + self.dic[i]["name"] + """_""" + value + """_""" + self.dic[i][
                        'property_id'] + """_""" + str(key)
                for key, value in self.dic[i]["ref_dic"].items():
                    query += """ ?""" + self.dic[i]["name"] + """_ref_""" + self.dic[i]['property_id'] + """_""" + str(
                        key)
                query += """ ?""" + self.dic[i]["name"] + """_rank_""" + self.dic[i]['property_id'] + """_rank"""
            else:
                query += """ ?""" + self.dic[i]["name"] + """_""" + self.dic[i]['property_id']
                if self.dic[i]["label"]:
                    query += """ ?""" + self.dic[i]["name"] + """_""" + self.dic[i]['property_id'] + """Label"""
        query += """ WHERE {""" + rdf_triple + time_filter + label_statement + """} """ + limit_statement
        return query

    def search_property_for_verbose(self):
        property_to_name = {}
        ref_to_name = {}
        rdf_triple, time_filter, limit_statement = """""", """""", """"""
        if self.dic[self.count]["rowVerbose"] or self.dic[self.count]["colVerbose"]:
            for i in range(1, self.count):
                if self.dic[i]["focus"] == "Entity ID":
                    if self.dic[i]["isSubject"]:
                        rdf_triple += """?""" + self.dic[i]["name"] + """ wdt:""" + self.dic[i][
                            "property_id"] + """ wd:""" + self.entity_id + """ ."""
                    else:
                        rdf_triple += """wd:""" + self.entity_id + """ wdt:""" + self.dic[i]["property_id"] + """ ?""" + \
                                      self.dic[i]["name"] + """ ."""
                else:
                    last = self.dic[i]["focus"].rfind('_')
                    focus = self.dic[i]["focus"][:last]
                    if self.dic[i]["isSubject"]:
                        rdf_triple += """?""" + self.dic[i]["name"] + """ wdt:""" + self.dic[i][
                            "property_id"] + """ ?""" + focus + """ ."""
                    else:
                        rdf_triple += """?""" + focus + """ wdt:""" + self.dic[i][
                            "property_id"] + """ ?""" + self.dic[i]["name"] + """ ."""
            if self.dic[self.count]["focus"] == "Entity ID":
                rdf_triple += """wd:""" + self.entity_id + """ p:""" + self.dic[self.count][
                    'property_id'] + """ ?statement.""" + \
                              """?statement """ + """ps:""" + self.dic[self.count]['property_id'] + """ ?item.""" + \
                              """?statement """ + """?pq """ + """?obj.""" + \
                              """?qual wikibase:qualifier ?pq.""" + \
                              """OPTIONAL{ ?statement prov:wasDerivedFrom ?refnode. ?refnode ?pr ?r.}"""
            else:
                last = self.dic[self.count]["focus"].rfind('_')
                focus = self.dic[self.count]["focus"][:last]
                rdf_triple += """?""" + focus + """ p:""" + self.dic[self.count][
                    'property_id'] + """ ?statement.""" + \
                              """?statement """ + """ps:""" + self.dic[self.count]['property_id'] + """ ?item.""" + \
                              """?statement """ + """?pq """ + """?obj.""" + \
                              """?qual wikibase:qualifier ?pq.""" + \
                              """OPTIONAL{ ?statement prov:wasDerivedFrom ?refnode. ?refnode ?pr ?r.}"""
        if self.time_property is not None:
            time_filter = """?""" + self.dic[1]["name"] + """ p:""" + self.time_property + """ ?pubdateStatement.	
                                  ?pubdateStatement ps:""" + self.time_property + """ ?date	
                                  FILTER (YEAR(?date) = """ + self.time + """)"""
        if self.limit is not None:
            limit_statement = """LIMIT """ + str(self.limit)
        label_statement = """Service wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }"""
        query = """SELECT DISTINCT """
        if self.dic[self.count]["rowVerbose"] or self.dic[self.count]["colVerbose"]:
            query += """?item""" + """ ?qual""" + """ ?qualLabel""" + """ ?obj """ + """?pr ?prLabel"""
            query += """ WHERE {""" + rdf_triple + time_filter + label_statement + """} """ + limit_statement
            query_result = get_results(endpoint_url, query)
            for result in query_result['results']['bindings']:
                if 'qual' in result:
                    property_to_name[result['qual']['value'].split('/')[-1]] = result['qualLabel']['value'].replace(' ',
                                                                                                                    '_')
                if 'pr' in result:
                    ref_to_name[result['pr']['value'].split('/')[-1]] = result['prLabel']['value'].replace(' ', '_')
        else:
            query += """?""" + self.dic[self.count]["name"] + """ """
        return property_to_name, ref_to_name

    def __str__(self):
        return str(self.df)

    def __getattr__(self, col_name):
        if col_name in self.df.columns:
            return self.df[col_name]
        else:
            print(col_name + " has not been found.")
            return None


def createRelation(entity_id: str, property_id=None, isSubject=None, rowVerbose=None, colVerbose=None,
                   time_property=None, time=None, name=None, label=False, limit=None):
    if property_id and not name:
        print("Please specify the name of the first column")
        return None
    return Relation(entity_id, property_id, isSubject, rowVerbose, colVerbose, time_property, time, name, label, limit)

def get_Firstname(name: str):
    return name.split(' ')[0]

def get_Lastname(name: str):
    return name.split(' ')[-1]

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


def get_name(id: str):
    query = """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 	
                PREFIX wd: <http://www.wikidata.org/entity/> 	
                select  *	
                where {	
                wd:""" + id + """ rdfs:label ?label .	
                FILTER (langMatches( lang(?label), "EN" ) )	
                } 	
                LIMIT 1"""
    results = get_results(endpoint_url, query)
    result = ''
    for res in results["results"]["bindings"]:
        result = res['label']['value']
    return result


You should consider upgrading via the '/Users/jack/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
print("20 Singers")
r = createRelation("Q177220")
r.extend("P106", True, "Singer", limit=20)
r.changeFocus("Singer_P106")
r.extend("P569", False, "Date_of_Birth")
r.query()
r.df

20 Singers


,Entity ID,Singer_P106,Date_of_Birth_P569
0,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q18821,1170-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q19417,1969-09-07T00:00:00Z
2,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q19734,1971-01-07T00:00:00Z
3,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q23880,1968-09-19T00:00:00Z
4,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q23543,1971-04-16T00:00:00Z
5,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q21087,1971-12-02T00:00:00Z
6,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q18735,1967-12-08T00:00:00Z
7,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q25679,1967-10-28T00:00:00Z
8,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q15079,1967-09-12T00:00:00Z
9,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q19845,1972-08-06T00:00:00Z


In [3]:
def age_cal(birth):
    if birth == 'NA':
        return 'NA'
    date_time_obj = datetime.datetime.strptime(birth[:4], '%Y')
    return datetime.datetime.now().year - date_time_obj.year

r.applyFunction('Date_of_Birth_P569',age_cal,'Age')

r.df

,Entity ID,Singer_P106,Date_of_Birth_P569,Age
0,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q31152,1897-09-10T00:00:00Z,123
1,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q39282,1755-08-31T00:00:00Z,265
2,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q26834,NA,NA
3,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q32529,1969-11-12T00:00:00Z,51
4,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q31768,1970-04-23T00:00:00Z,50
5,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q40890,1970-12-14T00:00:00Z,50
6,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q40096,1968-09-25T00:00:00Z,52
7,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q26625,1968-09-16T00:00:00Z,52
8,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q34584,1968-09-17T00:00:00Z,52
9,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q39983,1971-10-26T00:00:00Z,49


In [2]:
print("20 Singers")
r = createRelation("Q177220")
r.extend("P106", True, "Singer", limit=20, label=True)
r.changeFocus("Singer_P106")
r.extend("P569", False, "Date_of_Birth")
r.query()
r.df

20 Singers


,Entity ID,Singer_P106,Singer_P106Label,Date_of_Birth_P569
0,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q15897,Amy Winehouse,1983-09-14T00:00:00Z
1,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q16297,William Shatner,1931-03-22T00:00:00Z
2,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q22844,Huh Gak,1984-11-15T00:00:00Z
3,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q19848,Lady Gaga,1986-03-28T00:00:00Z
4,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q22979,Daniela Katzenberger,1986-10-01T00:00:00Z
5,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q25554,Marcus Mumford,1987-01-31T00:00:00Z
6,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q16781,Jacky Cheung,1961-07-10T00:00:00Z
7,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q25660,Tomasz Wróblewski,1980-06-02T00:00:00Z
8,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q16472,Robert Mitchum,1917-08-06T00:00:00Z
9,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q14541,Terry Callier,1945-05-24T00:00:00Z


In [3]:
r.applyFunction('Singer_P106Label','F1','First Name')
r.applyFunction('Singer_P106Label','F2','Last Name')

r.df

,Entity ID,Singer_P106,Singer_P106Label,Date_of_Birth_P569,First Name,Last Name
0,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q15897,Amy Winehouse,1983-09-14T00:00:00Z,Amy,Winehouse
1,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q16297,William Shatner,1931-03-22T00:00:00Z,William,Shatner
2,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q22844,Huh Gak,1984-11-15T00:00:00Z,Huh,Gak
3,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q19848,Lady Gaga,1986-03-28T00:00:00Z,Lady,Gaga
4,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q22979,Daniela Katzenberger,1986-10-01T00:00:00Z,Daniela,Katzenberger
5,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q25554,Marcus Mumford,1987-01-31T00:00:00Z,Marcus,Mumford
6,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q16781,Jacky Cheung,1961-07-10T00:00:00Z,Jacky,Cheung
7,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q25660,Tomasz Wróblewski,1980-06-02T00:00:00Z,Tomasz,Wróblewski
8,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q16472,Robert Mitchum,1917-08-06T00:00:00Z,Robert,Mitchum
9,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q14541,Terry Callier,1945-05-24T00:00:00Z,Terry,Callier


In [4]:
func_lib.func_num()

4

In [5]:
def get_entity_id(url):
    return url.split('/')[-1]

func_lib.add_func(get_entity_id)

func_lib.func_num()

5

In [7]:
r.applyFunction('Entity ID','F4','id')

In [8]:
r.df

,Entity ID,Singer_P106,Singer_P106Label,Date_of_Birth_P569,First Name,Last Name,id
0,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q2680,Bruce Willis,1955-03-19T00:00:00Z,Bruce,Willis,Q177220
1,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q8741,Rüdiger Skoczowsky,1986-01-01T00:00:00Z,Rüdiger,Skoczowsky,Q177220
2,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q3008,Mike Oldfield,1953-05-15T00:00:00Z,Mike,Oldfield,Q177220
3,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q1698,Serge Gainsbourg,1928-04-02T00:00:00Z,Serge,Gainsbourg,Q177220
4,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q10681,Alan Stivell,1944-01-06T00:00:00Z,Alan,Stivell,Q177220
5,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q1203,John Lennon,1940-10-09T00:00:00Z,John,Lennon,Q177220
6,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q1453,Alexandra Stan,1989-06-10T00:00:00Z,Alexandra,Stan,Q177220
7,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q4826,Rosa Valetti,1878-03-17T00:00:00Z,Rosa,Valetti,Q177220
8,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q1225,Bruce Springsteen,1949-09-23T00:00:00Z,Bruce,Springsteen,Q177220
9,http://www.wikidata.org/entity/Q177220,http://www.wikidata.org/entity/Q10695,Dan Ar Braz,1949-01-15T00:00:00Z,Dan,Braz,Q177220
